# Routing Topology: OSPF using FRRouting with Chameleon

This notebook is an example of how to create a FABRIC routing experiment topology comprising nodes at three different sites. Each site has a local layer 2 (Ethernet) network connecting a set of local nodes and one gateway router. The three gateway routers connect to each other and use the [FRRouting](https://frrouting.org/) protocol suite to deploy [OSPF](https://en.wikipedia.org/wiki/Open_Shortest_Path_First) dameons to propagate route updates across the topology.

You might be familiar with the [Quagga](https://www.quagga.net/) router suite.  FRRouting is based on Quagga but has a more active upstream community including many large companies working on cloud networking.


## Step 1:  Configure the Environment


In [ ]:
import os
import sys

module_path = os.path.abspath(os.path.join(f"{os.environ['HOME']}/work/PRUTH-FABRIC-Examples/fablib_local"))
if module_path not in sys.path:
    sys.path.append(module_path)
from fablib_custom.fablib_custom import *

from chameleon_utils.chameleon_config import *
load_chameleon_rc_environment(chameleon_rc_file=f"{os.environ['HOME']}/work/fablib_local_private_config/Chameleon-openrc.sh")


from chameleon_utils.chameleon_stitching import *
from chameleon_utils.chameleon_servers import *

from fablib_common_utils.utils import *

from performance_testing.iperf3 import *

from my_experiment.frrouting_experiment import FRRouting_Experiment

## Step 2: Import the FABLlib Library


In [ ]:
frr_experiment = FRRouting_Experiment('FRRouting_OSPF')

In [ ]:
#frr_experiment.load()
#frr_experiment.display()

## Step 3 (Optional): Query for Available Tesbed Resources and Settings

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [ ]:
try:
    frr_experiment.list_resources()

except Exception as e:
    print(f"Exception: {e}")

## Step 4: Create the Experiment Slice

The following creates private layer 2 networks on three sites including a OSPF gateway routers that propogate routes acrross the topology. 


In [ ]:

#time_stamp = datetime.now(tz=tz.tzutc()).strftime('%Y%m%d%H%M')
#ext=f'_pruth_{time_stamp}'

frr_sites = { 'dall': { 'site': 'DALL', 'facility': 'FABRIC', 'node_count': 2 },
              'salt': { 'site': 'SALT', 'facility': 'FABRIC', 'node_count': 2 },
              'chameleon': { 'site': 'STAR', 'facility': 'CHI@UC', 'node_count': 2 },
              'utah': { 'site': 'UTAH', 'facility': 'FABRIC', 'node_count': 2 }
            }
    
frr_links = { 'Link12': ('dall','salt'),
              'Link23': ('salt','chameleon'),
              'Link31': ('chameleon','dall'),
              'Link24': ('salt','utah')
            }
    


In [ ]:
frr_experiment.configure(frr_sites, frr_links)

In [ ]:
frr_experiment.deploy()

In [ ]:
#frr_experiment.load()
frr_experiment.display()

In [ ]:
import traceback
try:
    # Add routers
    routers = {}
    for name, data in frr_sites.items():
        routers[name] = frr_experiment.add_router(name=f'{name}_Router', site=data['site'], cores=32, ram=128, disk=10)
   
    # Add links between routers
    links = {}
    for name, data in frr_links.items():
        router_a_name, router_b_name = data
        router_a = routers[router_a_name]
        router_b = routers[router_b_name]
        
        routers[name] = frr_experiment.add_router_link(name=name, router1=router_a, router2=router_b, nic_model='NIC_Basic')
    
    # Add local networks and nodes
    for name, data in frr_sites.items():
        if data['facility'] == 'FABRIC':
            frr_experiment.add_local_network(name=f'{name}_local_net', router=routers[name], node_count=data['node_count'], cores=4, ram=16, disk=10)
        elif data['facility'] == 'CHI@UC':
            frr_experiment.add_chameleon_local_network(name=f'{name}_local_net', router=routers[name], node_count=data['node_count'], verbose=True)
        else:
            print(f"Unknown facility")
            
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()


In [ ]:
frr_experiment.display()

In [ ]:
try:
    frr_experiment.submit()
    
except Exception as e:
    print(f"Slice Fail: {e}")
    traceback.print_exc()

In [ ]:
try:
    #frr_slice = FRRouting_Slice(slice_name)
    
    frr_experiment.load()
except Exception as e:
    print(f"Exception: {e}")
    raise e

In [ ]:
try:
    frr_experiment.wait_jupyter()
except Exception as e:
    print(f"Exception: {e}")

In [ ]:

#frr_experiment.load()
#print(f"{frr_experiment.nodes}")
#print(json.dumps(frr_experiment.nodes, indent=4))
frr_experiment.execute_on_all_edge_nodes(f'rm -rf fabric_node_tools')
frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
frr_experiment.execute_on_all_edge_nodes(f'chmod +x fabric_node_tools/*.sh')


In [ ]:
try:
    frr_experiment.slice.save('ospf_slice.graphml')
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:

print(json.dumps(frr_experiment.local_networks, indent=4))


In [ ]:
import traceback
try:
    frr_experiment.configure_devs()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
import traceback
try:
    for node in frr_experiment.slice.get_nodes():
        print(f'{node.get_name()}: {node.get_management_ip()}')
        
        #node.execute('ip addr list', quiet=False)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
import traceback
try:
    for node in frr_experiment.get_edge_nodes():
        print(f"node: {node.get_name()}")
        
    for router in frr_experiment.get_routers():
        print(f"router: {router.get_name()}")
        
    for local_networks in frr_experiment.get_local_networks():
        print(f"local_networks: {local_networks}")

    for router_links in frr_experiment.get_router_links():
        print(f"router_links: {router_links}")
 
    for local_network_name in frr_experiment.get_local_network_names():
        print(f"local_networks: {local_network_name}")

    for router_link_name in frr_experiment.get_router_link_names():
        print(f"router_links: {router_link_name}")
 
    
    frr_experiment.configure_routers()
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
#frr_experiment.upload_directory_to_all_edge_nodes('fabric_node_tools','.')
command = f'sudo yum install -y -q iperf3 iproute-tc && sudo ./fabric_node_tools/host_tune_redhat.sh'


frr_experiment.execute_on_all_edge_nodes(f'{command}')

## Step 8: Run the Experiment

We will just test `ping` RTT and look at `tracepath`. Your experiment should be more interesting!

Notice that if you run this quickly and repeatedly run this test against a specific target, you may see changes to the tracepath.  Initially the ping may even fail.  Why do you think this is happening?


In [ ]:
try:
    print(frr_experiment.nodes)
    target_node_name =  frr_experiment.nodes[0]['name'] # f'{site1}_local_net1'
    target_node = frr_experiment.slice.get_node(name=target_node_name)
    target_ip = frr_experiment.nodes[0]['data_plane_ip']
    
    threads = {}
    for node in frr_experiment.slice.get_nodes():
        print(f"Testing target node: {node.get_name()}, target IP: {target_ip}")

        threads[node] = node.execute_thread(f'ping -c 5 {target_ip}')
        
    for node,thread in threads.items():
        print(f"Result for {node.get_name()}, target IP: {target_ip}")
        stdout, stderr = thread.result()
        print (stdout, stderr)

        #stdout, stderr = node.execute(f'tracepath {target_ip}')
        #print (stdout, stderr)

except Exception as e:
    print(f"Exception: {e}")

In [ ]:
for source_node in frr_experiment.nodes:
    for target_node in frr_experiment.nodes:
        
        if source_node['name'] == target_node['name']:
            continue
            
        frr_experiment.iperf3_run(source_node=source_node, 
                                  target_node=target_node, 
                                  w='32m', P=16, t=20, i=10, O=10, verbose=False)
    
    

In [ ]:
frr_experiment.iperf3_run(source_node=frr_experiment.get_node('Site1_local_net1'), 
                                  target_node=frr_experiment.get_node('Site2_local_net1'), 
                                  w='32m', P=16, t=20, i=10, O=10, verbose=True)

In [ ]:
import traceback

try:
    iperf3_process_output(verbose=True)

except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

In [ ]:
frr_experiment.load()

#frr_experiment.display_init()
frr_experiment.display()

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

## Step 9: Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
try:
    frr_experiment.fablib.delete_all()
    #slice.delete()
except Exception as e:
    print(f"Exception: {e}")